In [1]:
from system_prompt import DEFAULT_SYSTEM_PROMPT

from dotenv import load_dotenv 

from typing import Annotated, Sequence, List, Optional
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [2]:
load_dotenv()

True

In [3]:
config = {
    "system_prompt": DEFAULT_SYSTEM_PROMPT,
    "temperature": 0.9,
    "max_output_tokens": 1008,
    "model": "gemini-2.5-flash-lite"
}

In [4]:
class AgentState(TypedDict):
    """Represents the state of the agent's workflow."""
    # A list of messages, annotated with a reducer to append new messages
    messages: Annotated[Sequence[BaseMessage], add_messages]
    
    # ADD THESE THREE FIELDS
    pending_transaction_params: Optional[dict]
    pending_transaction_type: Optional[str]
    is_confirmed: Optional[bool]

In [5]:
class InstructionStep(TypedDict):
    title: str
    details: List[str]

In [6]:
def _execute_swap(amount: float, from_token: str, to_token: str, slippage: float, gas_fee_estimate: str):
    """The actual logic to perform the swap (now an internal function)"""
    return {
        "status": "success",
        "tx_hash": "0xmockswap123",
        "details": f"Swapped {amount} {from_token} to {to_token} with {slippage}% slippage. Est. Fee: {gas_fee_estimate}"
    }

def _execute_send(amount: float, token: str, recipient: str, gas_fee_estimate: str):
    """The actual logic to send tokens (now an internal function)"""
    return {
        "status": "success",
        "tx_hash": "0xmocksend456",
        "details": f"Sent {amount} {token} to {recipient}. Est. Fee: {gas_fee_estimate}"
    }

In [7]:
@tool
def propose_swap(amount: float, from_token: str, to_token: str, slippage: float = 1.0, gas_fee_estimate: str = "10 USD"):
    """Propose a token swap transaction. This tool only returns a confirmation message; it does NOT execute the swap."""
    return f"Proposing swap of {amount} {from_token} for {to_token} with {slippage}% slippage (Est. Fee: {gas_fee_estimate})."

@tool
def propose_send(amount: float, token: str, recipient: str, gas_fee_estimate: str = "5 USD"):
    """Propose a token send transaction. This tool only returns a confirmation message; it does NOT execute the send."""
    return f"Proposing send of {amount} {token} to {recipient} (Est. Fee: {gas_fee_estimate})."

# Map to easily access execution functions later
EXECUTION_MAP = {
    "swap": _execute_swap,
    "send": _execute_send,
}

@tool
def give_instructions(
    title: str,
    summary: str,
    steps: List[InstructionStep],
    notes: Optional[List[str]] = None,
    warnings: Optional[List[str]] = None,
    checklist: Optional[List[str]] = None,
) -> str:
    """Generate step-by-step instructions in markdown format with optional notes, warnings, and checklist."""
    
    md = f"## {title}\n"
    md += f"**Short summary:** {summary}\n\n"
    md += "### Steps\n"
    for i, step in enumerate(steps, 1):
        md += f"{i}. **{step['title']}**\n"
        for d in step["details"]:
            md += f"   - {d}\n"
        md += "\n"

    if notes:
        for n in notes:
            md += f"> **Note:** {n}\n"
    if warnings:
        for w in warnings:
            md += f"> **Warning:** {w}\n"
    if checklist:
        md += "\n**Checklist**\n"
        for c in checklist:
            md += f"✅ {c}\n"

    return md

tools = [propose_swap, propose_send, give_instructions]

In [8]:
llm = ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_output_tokens=config["max_output_tokens"],
    model_kwargs={"system_instruction": config["system_prompt"]}
).bind_tools(tools)

In [9]:
MAX_CONTEXT = 16

def propose_node(state: AgentState) -> AgentState:

    # Keep only the last N messages
    messages = list(state["messages"])[-MAX_CONTEXT:]

    # Call LLM with ONLY the trimmed history.
    # The bound LLM already knows the system prompt and tools.
    response = llm.invoke(messages) 

    # Check for tool call (a proposal)
    if response.tool_calls:
        # Assuming one tool call for simplicity
        call = response.tool_calls[0]

        # 1. Save the proposal details to the state
        tx_type = call['name'].replace('propose_', '')
        tx_params = call['args']
        
        # 2. Build the explicit confirmation message for the user
        confirmation_msg = f"**CONFIRMATION REQUIRED**\n\n"
        confirmation_msg += f"**Action:** {tx_type.upper()}\n"
        
        # Iterate over parameters and format nicely
        for k, v in tx_params.items():
            confirmation_msg += f"- **{k.replace('_', ' ').title()}:** {v}\n"
        
        confirmation_msg += "\n**Reply 'YES' to confirm or 'NO' to cancel.**"

        # 3. Return the new state
        return {
            "messages": [response, AIMessage(content=confirmation_msg)], # Show tool call and confirmation prompt
            "pending_transaction_params": tx_params,
            "pending_transaction_type": tx_type,
            "is_confirmed": False # Explicitly set for tracking
        }

    # If no tool call, just return the chat response
    return {"messages": [response]}

# IMPORTANT: RENAME your old `swap_agent` function call to `propose_node` in the graph setup

In [10]:
def route_action(state: AgentState) -> str:
    # 1. Check if a transaction is pending in the state
    if state.get("pending_transaction_params"):
        last_message = state["messages"][-1]
        
        # 2. Check the user's latest reply (for YES/NO)
        if isinstance(last_message, HumanMessage):
            reply = last_message.content.strip().upper() 
            
            if reply == "YES":
                return "execute" # Go to execution node
            elif reply == "NO":
                return "cancel" # Go to cancel node

        # If a transaction is pending but the user hasn't replied YES/NO, 
        # we route to the `propose_node` which will re-print the confirmation prompt.
        return "propose_loop"
    
    # 3. If no transaction is pending, route to the LLM to process a new command
    # This acts as the entry for new user inputs.
    return "propose"

In [11]:
def execute_node(state: AgentState) -> AgentState:
    tx_type = state["pending_transaction_type"]
    tx_params = state["pending_transaction_params"]
    
    # 1. Run the execution function using the saved parameters
    result = EXECUTION_MAP[tx_type](**tx_params)
    
    # 2. Clear the pending state
    return {
        "messages": [AIMessage(content=message_content)],
        "pending_transaction_params": None,  # MUST BE CLEARED
        "pending_transaction_type": None,    # MUST BE CLEARED
        "is_confirmed": True
    }

def cancel_node(state: AgentState) -> AgentState:
    # Clear the pending state and send a message
    return {
        "messages": [AIMessage(content="Transaction canceled as requested.")],
        "pending_transaction_params": None,  # MUST BE CLEARED
        "pending_transaction_type": None,    # MUST BE CLEARED
        "is_confirmed": False
    }

In [12]:
# Create a new graph with the defined state
graph = StateGraph(AgentState)

# Add nodes
# RENAME the old "swap_agent" to "propose"
graph.add_node("propose", propose_node) 
# The old "tools" node is no longer needed

# ADD the new execution and cancel nodes
graph.add_node("execute", execute_node)
graph.add_node("cancel", cancel_node)


# Set entry point
graph.set_entry_point("propose")

# Define edges
# The routing is now done with the smart router
graph.add_conditional_edges(
    "propose",
    route_action, # Check user reply/state
    {
        "execute": "execute",     # If user replied YES
        "cancel": "cancel",       # If user replied NO
        "propose_loop": END,      # End the run to await new input (the YES/NO reply)
        "propose": END            # End if just a chat response (no tool call)
    },
)

# Execution and Cancel both lead to END
graph.add_edge("execute", END)
graph.add_edge("cancel", END)

# compile the graph
app = graph.compile()

In [13]:
def chat_with_agent(user_input: str, history=None):
    if history is None:
        history = []
    
    # --- STEP 1: Capture the state BEFORE the user's input ---
    # The history list currently holds everything from PREVIOUS runs.
    
    # Store the user input and set the initial state
    history.append(HumanMessage(content=user_input))
    state = {"messages": history}
    
    # Keep track of the length *before* the run starts
    initial_length = len(history)
    
    # --- STEP 2: Invoke the graph ---
    result = app.invoke(state)
    
    # --- STEP 3: Get the new messages generated by the graph ---
    
    # The 'messages' key in the result is the *full* history list. 
    # New messages are those from the index `initial_length` onwards.
    
    # The new messages usually start with the LLM's response (AIMessage)
    # and might include a ToolMessage or a second AIMessage (like your confirmation prompt).
    new_messages = result["messages"][initial_length:]
    
    # --- STEP 4: Update the history with ALL new messages ---
    # Since the `app.invoke` returns the state with ALL messages, 
    # we can simply reassign `history` to the full message list from the result.
    history = result["messages"] # Reassign history to the full, updated list
    
    # Return the newly generated messages, which is what you want to output
    return history, new_messages

# We need to change the main loop to use this new return structure!

In [14]:
history = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    
    # 1. Call the function, capturing both the updated history and the NEW messages
    history, new_messages = chat_with_agent(user_input, history)
    
    # 2. Display the user's message (which was just input)
    print("---------------------------------------------")
    print(f"You: {user_input}") # Display the user's input
    print("---------------------------------------------")
    
    # 3. Iterate and display all new messages generated by the graph run
    for msg in new_messages:
        # Determine the prefix based on message type
        prefix = "AI:"
        if isinstance(msg, HumanMessage):
            # This should rarely happen but is a good safeguard
            prefix = "Human:" 
        elif isinstance(msg, ToolMessage):
            prefix = "Tool (Internal):"
            # Optional: You might want to skip printing tool results for a cleaner user experience
            # continue 
        
        # Print the message content
        print(f"{prefix} {msg.content}")
    
    # Add a separator for the next turn
    print("---------------------------------------------\n")

---------------------------------------------
You: hi
---------------------------------------------
AI: Hi there! How can I help you today?
---------------------------------------------

---------------------------------------------
You: I want to send 10usdc to mom
---------------------------------------------
AI: What is your mom's address?
---------------------------------------------

---------------------------------------------
You: 0x33ca4EEf9C5A865dd9254CdBa1398a72770004Cc
---------------------------------------------
AI: 
AI: **CONFIRMATION REQUIRED**

**Action:** SEND
- **Recipient:** 0x33ca4EEf9C5A865dd9254CdBa1398a72770004Cc
- **Token:** USDC
- **Amount:** 10

**Reply 'YES' to confirm or 'NO' to cancel.**
---------------------------------------------

---------------------------------------------
You: yes
---------------------------------------------
AI: I can only propose transactions. Please confirm this transaction in your wallet.
----------------------------------------

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Please ensure that function call turn comes immediately after a user turn or after a function response turn.